### Import bibliotek

In [1]:
import json
import os
import requests
from bs4 import BeautifulSoup

### Narzędzia

In [17]:
def extract(ancestor, selector = None, attribute = None, many = False):
    if selector:
        if many:
            if attribute:
                return [item[attribute].strip() for item in ancestor.select(selector)]
            return [item.text.strip() for item in ancestor.select(selector)]
        if attribute:
            try:
                return ancestor.select_one(selector)[attribute].strip()
            except TypeError:
                return None
        try:
            return ancestor.select_one(selector).text.strip()
        except AttributeError:
            return None     
    if attribute:
        try:
            return ancestor[attribute]
        except TypeError:
            return None
    return ancestor.text.strip()

In [21]:
review_scheme = {
    "review_id": (None, "data-entry-id"),
    "author": ("span.user-post__author-name",),
    "recomendation": ("span.user-post__author-recomendation > em",),
    "stars": ("span.user-post__score-count",),
    "content": ("div.user-post__text",),
    "pros": ("div.review-feature__item--positive", None, True,),
    "cons": ("div.review-feature__item--negative", None, True,),
    "likes": ("button.vote-yes > span",),
    "dislikes": ("button.vote-no > span",),
    "publish_date": ("span.user-post__published > time:nth-child(1)", 'datetime',),
    "purchase_date": ("span.user-post__published > time:nth-child(2)", 'datetime',),
            }

### Pobranie ze strony Ceneo.pl opinii o konkretnym produkcie


In [2]:
with open("./headers.json", "r")as jf:
    headers = json.load(jf)



In [22]:
product_id = "84514582"
next_page = f"https://www.ceneo.pl/{product_id}#tab=reviews"
all_reviews = []
while next_page:
    response = requests.get(next_page, headers=headers)
    print(next_page)
    if response.status_code == 200:
        page_dom = BeautifulSoup(response.text, "html.parser")
        reviews = page_dom.select("div.js_product-review:not(.user-post--highlight)")
        print(len(reviews))
        for review in reviews:
            single_review = {
                key: extract(review, *value)
                for key, value in review_scheme.items()
            }
            all_reviews.append(single_review)

        try:
            next_page = "https://www.ceneo.pl" + extract(page_dom, "a.pagination__next", "href")
        except TypeError:
            next_page = None
print(all_reviews)
        

https://www.ceneo.pl/84514582#tab=reviews
10
https://www.ceneo.pl/84514582/opinie-2
10
https://www.ceneo.pl/84514582/opinie-3
10
https://www.ceneo.pl/84514582/opinie-4
10
https://www.ceneo.pl/84514582/opinie-5
10
https://www.ceneo.pl/84514582/opinie-6
10
https://www.ceneo.pl/84514582/opinie-7
10
https://www.ceneo.pl/84514582/opinie-8
10
https://www.ceneo.pl/84514582/opinie-9
6
[{'review_id': '12535429', 'author': 'g...6', 'recomendation': None, 'stars': '3/5', 'content': 'Instalacja łatwa. Oprogramowanie HP Smart bezproblemowe. Zwrot 100zl Cashback z Promocji po kilku dniach od zakupu. Skanuje. Drukuje. \nPrzez pierwsze 2-3 dni wszystko w porządku, wydruki czarne i kolorowe bardzo dobre. Po 3 dniu, zupełnie nagle, drukarka zaczyna drukować białe poziome kreski na wydrukach kolorowych. W czarnym bardzo dobrze. Wydruki w najlepszej jakości są bardzo dobre. Wydruki w jakości normalnej i roboczej są moim zdaniem wadliwe. Kupiłem 2 drukarki. Jedną udało mi się zwrócić w ramach 14 zwrotu. Dr

### Zapisanie wszystkich opinii w bazie danych

In [4]:
if not os.path.exists("./opinions"):
    os.mkdir("./opinions")

In [5]:
with open(f"./opinions/{product_id}.json", "w", encoding = "UTF-8") as jf:
    json.dump(all_reviews, jf, indent=4, ensure_ascii=False)